# Fragrantica perfume review clasifier (LSTM with stopword removed)

In [3]:
import pandas as pd
import requests
from os import path
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
# Model Name
MODEL_NAME = 'lstm_with_lemmatizated_05'

# HyperParameters
PAD_LEN = 200
NUM_WORDS = 5000
EMBEDDING = 200
BATCH_SIZE = 256


In [5]:
data_path = '../dataset/dataset_210626_215600.csv'
data_exist = path.exists(data_path)

if not data_exist:
    url = 'https://kyuuuw-nlp-dataset.s3.ap-northeast-2.amazonaws.com/fragrantica/dataset_210626_215600.csv'
    r = requests.get(url, allow_redirects=True)
    open(data_path, 'w').write(r.content)

In [6]:
data = pd.read_csv(data_path)

X_data = data['lemmatizated']
y_data = data['label']

print(len(X_data))
print(len(y_data))


##### 토큰화 및 인덱스 부여

74779
74779


In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(X_data[0])
print(len(sequences[0]))

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
76


##### 인덱스별 단어 표시

In [8]:
word_to_index = tokenizer.word_index

##### 빈도수 분석

In [9]:
threshold = 2
total_cnt = len(word_to_index) # 총 단어의 수
rare_cnt = 0 # 등장 빈도 수가 threshold 보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"등장 빈도가 {threshold}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합(vocabulary)에서 희귀 단어의 비율: {rare_cnt / total_cnt}" )
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {rare_freq / total_freq}")

등장 빈도가 2번 이하인 희귀 단어의 수: 43039
단어 집합(vocabulary)에서 희귀 단어의 비율: 0.5753877005347594
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.012692410682162147


##### 등장 횟수 상위 50000개의 word 만 사용

In [10]:
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(sequences[:5])

[[6, 72, 170, 54, 93, 1439, 14, 1, 25, 1027, 1892, 3407, 824, 888, 463, 107, 185, 31, 309, 39, 1190, 3248, 25, 2495, 2415, 3407, 1, 15, 6, 1472, 50, 4902, 80, 2482, 3, 158, 666, 255, 320, 123, 1181, 108, 67, 152, 1439, 621, 31, 263, 191, 4, 1, 1389, 336, 463, 413, 46, 108, 175, 303, 521, 218, 168, 44, 1690, 349, 312, 895, 432, 474, 39, 28, 350, 22, 179, 11], [21, 464, 121, 2941, 964, 1, 2, 1209, 25, 2, 10, 769, 494, 64, 8, 1, 659, 2873, 61, 188, 73, 33, 64, 37, 731, 8, 163, 8, 349, 1013, 1691, 1019, 492, 10, 52, 54, 802, 11, 1080, 783, 309, 39, 28, 25, 2941, 263, 1594, 424, 213, 117, 1140, 61, 10, 152, 775, 9, 24, 357, 1, 1, 152, 1031, 67, 139, 437, 44, 24, 223, 200, 38, 13, 659, 81, 83, 1350, 2941, 78, 135, 204, 263, 73, 439, 26, 4, 2941, 30, 1, 301, 14, 40, 235, 208, 97, 90, 1611, 8, 8, 254, 223, 1595, 1629, 30, 117, 43, 14, 759, 11, 335, 7, 335, 176, 177, 601, 1101, 22, 11, 2589, 243, 15, 121, 219, 13, 823], [10, 365, 63, 238, 129, 10, 138, 52, 32, 2326, 6, 448, 6, 72, 313, 1220, 49

##### Split data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(sequences, y_data)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

56084
56084
18695
18695


In [12]:
print(X_train[0])
print(y_train[0])

[49, 10, 1, 38, 48, 20, 384, 570, 1, 48, 200, 35, 1825, 355, 127, 6, 48, 260, 1]
2.0


In [13]:
# padding and trimming
X_train = pad_sequences(X_train, maxlen=PAD_LEN)
X_test = pad_sequences(X_test, maxlen=PAD_LEN)

In [14]:
# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[0])

[0. 0. 1. 0.]


In [15]:
model = Sequential()
model.add(Embedding(NUM_WORDS, EMBEDDING))
model.add(LSTM(EMBEDDING))
model.add(Dense(4, activation='softmax'))

model.summary()

[2021-06-26 18:28:41.241 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:682 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-06-26 18:28:41.267 tensorflow-2-3-gpu--ml-g4dn-xlarge-794be025f5602a375b1b7feb8a0a:682 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
dense (Dense)                (None, 4)                 804       
Total params: 1,321,604
Trainable params: 1,321,604
Non-trainable params: 0
_________________________________________________________________


In [16]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'../model/{MODEL_NAME}.h5', monitor='val_acc', mode='max')

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [18]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE, epochs=30, callbacks=[es, mc],
                    validation_data=(X_test, y_test))

Epoch 1/30
220/220 [==============================] - 16s 74ms/step - loss: 0.9522 - acc: 0.6117 - val_loss: 0.7584 - val_acc: 0.7179
Epoch 2/30
220/220 [==============================] - 16s 72ms/step - loss: 0.6582 - acc: 0.7564 - val_loss: 0.6871 - val_acc: 0.7415
Epoch 3/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5986 - acc: 0.7780 - val_loss: 0.7302 - val_acc: 0.7238
Epoch 4/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5637 - acc: 0.7900 - val_loss: 0.7185 - val_acc: 0.7310
Epoch 5/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5535 - acc: 0.7947 - val_loss: 0.7430 - val_acc: 0.7186
Epoch 6/30
220/220 [==============================] - 16s 73ms/step - loss: 0.5213 - acc: 0.8063 - val_loss: 0.7952 - val_acc: 0.7070
Epoch 00006: early stopping
